In [8]:
import json
import requests
from datetime import datetime, timedelta 
from lxml import etree
import json
from base64 import b64decode
import pandas as pd

# 从ixigua收集信息

In [27]:
user_id_list = [4492956276, 51045089537]

In [28]:
def downloadfile(filepath, url):
    print('downloadfile %s' % (url))
    r=requests.get(url, stream=True)
    print("****Connected****")
    f=open(filepath,'wb');
    print("Donloading.....")
    for chunk in r.iter_content(chunk_size=255):
        if chunk: # filter out keep-alive new chunks
            f.write(chunk)
    print("Done")
    f.close()

def request_detail_page(path, item):
    cookies = {
        'wafid':'af2ade58-1768-4fea-983d-c2ad0cf69fc2',
        'wafid.sig': 'xXI_Dn80Pyi_4TsddkTjVMYvnAQ',
        'xiguavideopcwebid': '6798376759479977479',
        'xiguavideopcwebid.sig': 'reKWpJ4q1-zRPqWZ3roOINB3wGc'
    }


    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7,ko;q=0.6,zh-TW;q=0.5',
        'cache-control': 'max-age=0',
        'referer': 'https://www.ixigua.com/',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
    }

    print('https://www.ixigua.com%s' % (path))
    r = requests.get('https://www.ixigua.com%s' % (path), headers=headers, cookies=cookies)
    html = etree.HTML(r.text)
    str_list = html.xpath('.//script/text()')
    for one in str_list:
        if 'video_1' in one:
            content = one.replace('window._SSR_HYDRATED_DATA=', '')

            o = json.loads(content)
            video_id = o['Projection']['video']['videoResource']['normal']['video_id']
            video_url = b64decode(o['Projection']['video']['videoResource']['normal']['video_list']['video_3']['main_url']).decode()

            downloadfile('%s.mp4' % (video_id), video_url)

            item['video_id'] = video_id
            return item

In [31]:
def main():
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7,ko;q=0.6,zh-TW;q=0.5',
        'cache-control': 'max-age=0',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1'
    }

    results = []
#     ct = datetime.now() - timedelta(days = 1) 
    ct = datetime.now()
    for user_id in user_id_list:
        min_behot_timestamp = 0
        for i in range(0, 3):
            url = 'https://m.ixigua.com/video/app/user/home/?to_user_id=%d&format=json&max_behot_time=%d' % (user_id, min_behot_timestamp)

            r = requests.get(url, headers=headers)
            o = json.loads(r.text)

            if not 'data' in list(o.keys()) or o['data'] is None or len(o['data']) == 0:
                continue

            min_behot_timestamp = o['data'][0]['behot_time']
            for item in o['data']:
                timestamp = item['behot_time']
                behot_time = datetime.fromtimestamp(timestamp) 

                if min_behot_timestamp > timestamp:
                    min_behot_timestamp = timestamp

                if behot_time.month == ct.month and behot_time.day == ct.day and behot_time.year == ct.year:
                    
                    try:
                        vitem = request_detail_page('/i%s' % (item['item_id']), item)
                        results.append(vitem)
                    except BaseException as e:
                        print(e)

    return results


In [32]:
data = main()

https://www.ixigua.com/i6799544496053813768
downloadfile http://v3-tt.ixigua.com/bf24127b52a268c3afad9bc0bc61aa3b/5e5cecd7/video/tos/cn/tos-cn-ve-4/75387bf424904f4084fe40147a4db2b1/?a=1768&br=0&bt=719&cr=0&cs=0&dr=0&ds=3&er=&l=202003021819470100140181350BA8CCF6&lr=&qs=0&rc=ajluPDp0eDRnczMzaTczM0ApNGlkNDw1aDtkN2U3PDxnaWdhX2wvXzZjXmJfLS1hLS9zc18yLl5eNl80LmNeLy4tMC46Yw%3D%3D&vl=&vr=
****Connected****
Donloading.....
Done
https://www.ixigua.com/i6799522222676378123
downloadfile http://v3-tt.ixigua.com/fb828943668907ce06bcb3fa3906d912/5e5ceed6/video/tos/cn/tos-cn-ve-4/e1791fd3505540d2b1ec2454ea5b97d2/?a=1768&br=0&bt=349&cr=0&cs=0&dr=0&ds=3&er=&l=20200302181951010014018141179D69A0&lr=&qs=0&rc=M2xra3c6eDtmczMzNjczM0ApMzo4Ojg1ZmVoNzNlNTw5NWc1YzVgaWo2L2JfLS0zLS9zc2AvXi0tMS0zMl5gMF9fNTM6Yw%3D%3D&vl=&vr=
****Connected****
Donloading.....
Done
https://www.ixigua.com/i6799515194948059651
downloadfile http://v3-tt.ixigua.com/7e56b1f14a1b205dbe41b1233bf4c515/5e5ced1e/video/tos/cn/tos-cn-ve-4/f2f9955d

Done
https://www.ixigua.com/i6799521227405787655
downloadfile http://v3-tt.ixigua.com/d290029b6b84833b30c83fc1dc6a8271/5e5cec9b/video/tos/cn/tos-cn-ve-31/575e89b196ea4f08af51efda5aa37a5f/?a=1768&br=0&bt=1527&cr=0&cs=0&dr=0&ds=3&er=&l=2020030218203301019409901934AD7978&lr=&qs=0&rc=M3F0bm90NzRmczMzN2kzM0ApPDdmZDRkaGU5NzVlaTwzNmc0ZWxeX3FiLmJfLS1fLi9zcy1hLy1eYGExLWMzLmI0MGA6Yw%3D%3D&vl=&vr=
****Connected****
Donloading.....
Done
https://www.ixigua.com/i6799519599789343240
downloadfile http://v3-tt.ixigua.com/00f054ccc4575a9761f40796ce13f94d/5e5cec3f/video/tos/cn/tos-cn-ve-31/d2f33b4e16be48ef80697c81123ddbd6/?a=1768&br=0&bt=1031&cr=0&cs=0&dr=0&ds=3&er=&l=202003021820410100120631670DA1650D&lr=&qs=0&rc=M2ZpOTc0ZTVlczMzN2kzM0ApM2g1ZzhlZGVnNzs8ZWQ6NWdoL29pZmxvcmJfLS0vLi9zczA0YDNiYTMzNi4uMDUxMWM6Yw%3D%3D&vl=&vr=
****Connected****
Donloading.....
Done
https://www.ixigua.com/i6799518473467724295
downloadfile http://v3-tt.ixigua.com/fdbd713207ddd2de611df82eb2c943b0/5e5cec89/video/tos/cn/tos-cn-ve-3

Done
https://www.ixigua.com/i6799472984223384078
downloadfile http://v3-tt.ixigua.com/83e95b8dad06842617f1727caec5c703/5e5ced5b/video/tos/cn/tos-cn-ve-31/e37ae60950804173a36f531a3e9de303/?a=1768&br=0&bt=550&cr=0&cs=0&dr=0&ds=3&er=&l=2020030218212001019409902119AF3BEC&lr=&qs=0&rc=M2xuZGY1Zjw8czMzNGkzM0ApZjNpZTtlZzw0NzM6OGg2ZWdqZGtfMG0wNmJfLS01Li9zcy00YC4wXy00XzYwMGAzX146Yw%3D%3D&vl=&vr=
****Connected****
Donloading.....
Done
https://www.ixigua.com/i6799468907749966349
downloadfile http://v3-tt.ixigua.com/f565f5e7b8452cf7fb7dca16bd09aa28/5e5cec5f/video/tos/cn/tos-cn-ve-31/4d07e6bce09c4782a85e26f5c9f2f5dd/?a=1768&br=0&bt=434&cr=0&cs=0&dr=0&ds=3&er=&l=2020030218212301001708721100A13F12&lr=&qs=0&rc=anE6OTN4b3Y8czMzN2kzM0ApZGc2OTM6OjtoNzw3ZzQ8O2djXnBval8xLmJfLS0yLi9zczBjMzZjMC1eYl40LzNhL2E6Yw%3D%3D&vl=&vr=
****Connected****
Donloading.....
Done
https://www.ixigua.com/i6799468290629435917
downloadfile http://v3-tt.ixigua.com/ea0a2aab44c87019d4537ce4e0ba0d03/5e5cec68/video/tos/cn/tos-cn-ve-31/

In [55]:
data[0]

{'log_pb': {'is_ad_event': '0',
  'impr_id': '2020030218194701001404407612135841',
  'is_following': '0',
  'group_source': '2',
  'author_id': '4492956276',
  'group_id': '6799544496053813768'},
 'verify_reason': '',
 'media_name': '央视新闻',
 'video_user_like': 0,
 'ban_comment': 0,
 'pread_params': {'rec_quality': 0,
  'media_id': 0,
  'item_id': 6799544496053813768,
  'category_tag': 'news',
  'from_category': '',
  'channel_id': 0,
  'display_flags': 0,
  'group_source': 2,
  'is_gov_article': False,
  'video_duration': 260,
  'group_id': 6799544496053813768,
  'review_comment_mode': 0,
  'categories': ['news']},
 'abstract': '你们奋战在前线，我们坚守在后方，为大家也为小家。说出你的心里话，说给你心中的Ta，有多少牵挂就有多少叮咛有多少思念就有多少嘱咐……说给你听 (总台央视记者 王新宇 谭海梅 杨雨桐 高涛 文豪 耿晓斌 许辉 赵振凯)',
 'impression_count': 0,
 'display_title': '',
 'ban_bury': 0,
 'article_type': 0,
 'tag': 'news',
 'has_m3u8_video': 0,
 'keywords': '',
 'play_auth_token': 'HMAC-SHA1:2.0:1583317187163747481:bab42eac5b9e4a8eb25a91fc371ad533:45BumxVJVxC0bBTrUSwhMOnnkuM=

In [56]:
tmp = []
for partial in data:
    if partial['title'] in ['央视新闻面对面：专访武汉日记拍摄者小林', '主播说联播丨武汉社区干部被一句“谢谢”弄哭']:
        continue
        
    tmp.append({
        'video_id': partial['video_id'],
        'abstract': partial['abstract'],
        'title': partial['title'],
        'media_name': partial['media_name']
    })

df = pd.DataFrame(tmp)
df

,video_id,abstract,title,media_name
0,v02004df0000bpeda1fu9qb729okb6dg,你们奋战在前线，我们坚守在后方，为大家也为小家。说出你的心里话，说给你心中的Ta，有多少牵挂...,我想对你说……,央视新闻
1,v02004630000bpec289um7ltch8hfi80,北京市纪委监委党风政风监督室主任肖飒通报：2月22日0点58分，黄某英所乘坐京牌小轿车从大兴...,黄某英是如何进京的？,央视新闻
2,v020040f0000bpebllnrri6cq24d34ig,,北京公安局：正调查黄某英事件是否构成违法犯罪,央视新闻
3,v02004930000bpebb058n75rc1du6vg0,,北京市回应黄某英离汉进京事件,央视新闻
4,v02004930000bpea8qg1ahq1v36dnd3g,火神山医院的陈静，是重症监护病房的“大管家”。她总是“爱唠叨”，每一个细节都会提醒病人。由于...,暖心又可爱！护士制作超萌新冠护患沟通手册,央视新闻
5,v02004400000bpe95j7iv426m4rmj390,国防部新闻发言人吴谦：我国决定把生物安全纳入国家安全体系，系统规划国家生物安全风险防控和治理...,国防部：中国将尽快出台生物安全法,央视新闻
6,v02004930000bpe7dmv3cp561vj4k5ng,军委后勤保障部卫生局局长陈景元：从1月24日开始，军队先后派出3批共4000多名医护人员驰援...,致敬！军队1万余名医护人员投入一线救治,央视新闻
7,v02004c60000bpe792egncoeue8kdhe0,国防部新闻发言人吴谦：中国汉字中的“人”字是一个相互支撑的架构。岁寒知松柏，患难见真情，当中...,吴谦：中国汉字的人是相互支撑的架构,央视新闻
8,v02004310000bpe72abrh3p817c8hnlg,军委后勤保障部卫生局局长陈景元：截至昨天，武汉火神山医院累计收治新冠肺炎患者1597例，已治...,火神山医院共收治患者1597例,央视新闻
9,v0201f630000bpedl1e8qbl46mh51ae0,近日，湖北武汉。“W大武汉紧急救援队”由四名志愿者组成，队长是一名88年的女孩，主要负责为留...,摆渡人丨他们，帮22个孕妇跨过“生门”,澎湃新闻


In [57]:
t = df[df['video_id'] == 'v020040f0000bpebllnrri6cq24d34ig']

index = 0
t.iloc[index]['title'], t.iloc[index]['abstract'], t.iloc[index]['video_id']

('北京公安局：正调查黄某英事件是否构成违法犯罪', '', 'v020040f0000bpebllnrri6cq24d34ig')

# 上传到Youtube

In [0]:
import http
import httplib2
import os
import random
import sys
import time

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage
from oauth2client.tools import argparser, run_flow


# Explicitly tell the underlying HTTP transport library not to retry, since
# we are handling retry logic ourselves.
httplib2.RETRIES = 1

# Maximum number of times to retry before giving up.
MAX_RETRIES = 10

# Always retry when these exceptions are raised.
RETRIABLE_EXCEPTIONS = (httplib2.HttpLib2Error, IOError, http.client.NotConnected,
  http.client.IncompleteRead, http.client.ImproperConnectionState,
  http.client.CannotSendRequest, http.client.CannotSendHeader,
  http.client.ResponseNotReady, http.client.BadStatusLine)

# Always retry when an apiclient.errors.HttpError with one of these status
# codes is raised.
RETRIABLE_STATUS_CODES = [500, 502, 503, 504]

# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
# client_secret. You can acquire an OAuth 2.0 client ID and client secret from
# the Google API Console at
# https://console.developers.google.com/.
# Please ensure that you have enabled the YouTube Data API for your project.
# For more information about using OAuth2 to access the YouTube Data API, see:
#   https://developers.google.com/youtube/v3/guides/authentication
# For more information about the client_secrets.json file format, see:
#   https://developers.google.com/api-client-library/python/guide/aaa_client_secrets
CLIENT_SECRETS_FILE = "client_secrets.json"

# This OAuth 2.0 access scope allows an application to upload files to the
# authenticated user's YouTube channel, but doesn't allow other types of access.
YOUTUBE_UPLOAD_SCOPE = "https://www.googleapis.com/auth/youtube.upload"
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

# This variable defines a message to display if the CLIENT_SECRETS_FILE is
# missing.
MISSING_CLIENT_SECRETS_MESSAGE = "Missing client secrets message"

VALID_PRIVACY_STATUSES = ("public", "private", "unlisted")


def get_authenticated_service():
  # flow = flow_from_clientsecrets(CLIENT_SECRETS_FILE,
  #   scope=YOUTUBE_UPLOAD_SCOPE,
  #   message=MISSING_CLIENT_SECRETS_MESSAGE)

  # storage = Storage("%s-oauth2.json" % sys.argv[0])
  # credentials = storage.get()

  # if credentials is None or credentials.invalid:
  #   credentials = run_flow(flow, storage, args)

  flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        CLIENT_SECRETS_FILE, YOUTUBE_UPLOAD_SCOPE)
  credentials = flow.run_console()

  # return build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
  #   http=credentials.authorize(httplib2.Http()))
  return googleapiclient.discovery.build(
        YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, credentials=credentials)

def initialize_upload(youtube, options):
  tags = None
  if options.keywords:
    tags = options.keywords.split(",")

  body=dict(
    snippet=dict(
      title=options.title,
      description=options.description,
      tags=tags,
      categoryId=options.category
    ),
    status=dict(
      privacyStatus=options.privacyStatus
    )
  )

  # Call the API's videos.insert method to create and upload the video.
  insert_request = youtube.videos().insert(
    part=",".join(body.keys()),
    body=body,
    # The chunksize parameter specifies the size of each chunk of data, in
    # bytes, that will be uploaded at a time. Set a higher value for
    # reliable connections as fewer chunks lead to faster uploads. Set a lower
    # value for better recovery on less reliable connections.
    #
    # Setting "chunksize" equal to -1 in the code below means that the entire
    # file will be uploaded in a single HTTP request. (If the upload fails,
    # it will still be retried where it left off.) This is usually a best
    # practice, but if you're using Python older than 2.6 or if you're
    # running on App Engine, you should set the chunksize to something like
    # 1024 * 1024 (1 megabyte).
    media_body=MediaFileUpload(options.file, chunksize=-1, resumable=True)
  )

  resumable_upload(insert_request)

# This method implements an exponential backoff strategy to resume a
# failed upload.
def resumable_upload(insert_request):
  response = None
  error = None
  retry = 0
  while response is None:
    try:
      print("Uploading file...")
      status, response = insert_request.next_chunk()
      if response is not None:
        if 'id' in response:
          print("Video id '%s' was successfully uploaded." % response['id'])
        else:
          exit("The upload failed with an unexpected response: %s" % response)
    except HttpError as e:
      if e.resp.status in RETRIABLE_STATUS_CODES:
        error = "A retriable HTTP error %d occurred:\n%s" % (e.resp.status,
                                                             e.content)
      else:
        raise
    except RETRIABLE_EXCEPTIONS as e:
      error = "A retriable error occurred: %s" % e

    if error is not None:
      print(error)
      retry += 1
      if retry > MAX_RETRIES:
        exit("No longer attempting to retry.")

      max_sleep = 2 ** retry
      sleep_seconds = random.random() * max_sleep
      print("Sleeping %f seconds and then retrying..." % (sleep_seconds))
      time.sleep(sleep_seconds)

In [30]:
argparser.add_argument("--file", required=True, help="Video file to upload")
argparser.add_argument("--title", help="Video title", default="Test Title")
argparser.add_argument("--description", help="Video description", default="Test Description")
argparser.add_argument("--category", default="22", help="Numeric video category. " + "See https://developers.google.com/youtube/v3/docs/videoCategories/list")
argparser.add_argument("--keywords", help="Video keywords, comma separated", default="")
argparser.add_argument("--privacyStatus", choices=VALID_PRIVACY_STATUSES, default=VALID_PRIVACY_STATUSES[0], help="Video privacy status.")

_StoreAction(option_strings=['--privacyStatus'], dest='privacyStatus', nargs=None, const=None, default='public', type=None, choices=('public', 'private', 'unlisted'), help='Video privacy status.', metavar=None)

In [31]:
youtube = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=898163829886-tn0jb3uua55fa2948lnbn5u9apt65npd.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.upload&state=9OhS3XXZ9UYdOWtt88ZNgpmdSF3vVQ&prompt=consent&access_type=offline
Enter the authorization code: 4/xAG7CFnUfP6HaJucT083lLZVZ79sbWXscPbyrZYXaz9hG2mgtfHxgUA


In [36]:
data

[{'abstract': '1月23日开始，13条vlog，小林走上街头走进社区，作为志愿者接送医务人员，为感染者送药，用个人视角记录着武汉人和病毒之间的拉锯战。他说，想让外面的人看到，在武汉生活的普通人正在非常积极勇敢地面对这些难。武汉，加油！',
  'aggr_type': 2,
  'article_sub_type': 0,
  'article_type': 0,
  'article_url': 'http://toutiao.com/group/6799254799951856140/',
  'ban_action': 0,
  'ban_bury': 0,
  'ban_comment': 0,
  'ban_danmaku': 0,
  'ban_download': 0,
  'behot_time': 1583074871,
  'bury_count': 0,
  'cell_type': 0,
  'comment_count': 30,
  'composition': 256,
  'cursor': 1583074871000,
  'danmaku_count': 0,
  'digg_count': 325,
  'display_title': '',
  'display_url': 'http://toutiao.com/group/6799254799951856140/',
  'group_flags': 32833,
  'group_id': 6799254799951856140,
  'group_id_str': '6799254799951856140',
  'group_source': 2,
  'has_image': False,
  'has_m3u8_video': 0,
  'has_mp4_video': 0,
  'has_video': True,
  'hot': 0,
  'image_count': 0,
  'image_list': [],
  'impression_count': 134215,
  'item_id': 6799254799951856140,
  'keywords': '',
  'level': 0,
  'log_pb': {'author

In [40]:
for record in data:
    params = [
        '--file=/content/%s.mp4' % (record['video_id']),
        '--title=%s' % (record['title']),
        '--description=%s' % (record['abstract']),
        '--category=25',
        '--keywords=News'
    ]
    print(params)

    if params[0] in ['--file=/content/v02004930000bpdsqb20ifkna9sj0oq0.mp4', '--file=/content/v020047e0000bpdsamvijqt1h4rr196g.mp4']:
        continue

    args = argparser.parse_args(args=params)
    
    try:
        initialize_upload(youtube, args)
    except HttpError as e:
        print(e)
        break
    #     print("An HTTP error %d occurred:\n%s" % (e.resp.status, e.content))

    # time.sleep(10)

['--file=/content/v02004930000bpdsqb20ifkna9sj0oq0.mp4', '--title=央视新闻面对面：专访武汉日记拍摄者小林', '--description=1月23日开始，13条vlog，小林走上街头走进社区，作为志愿者接送医务人员，为感染者送药，用个人视角记录着武汉人和病毒之间的拉锯战。他说，想让外面的人看到，在武汉生活的普通人正在非常积极勇敢地面对这些难。武汉，加油！', '--category=25', '--keywords=News']
['--file=/content/v020047e0000bpdsamvijqt1h4rr196g.mp4', '--title=主播说联播丨武汉社区干部被一句“谢谢”弄哭', '--description=今天是3月1日，《新闻联播》继续重点关注新冠肺炎疫情联防联控，头条聚焦武汉各个平凡岗位上的人们为抗击疫情作出的不懈努力，我们看看主播李梓萌想说些什么。', '--category=25', '--keywords=News']
['--file=/content/v020047e0000bpds70nijqt1h4rqtnlg.mp4', '--title=面对面丨他主动请缨新冠肺炎遗体解剖 不做点什么心里羞愧', '--description=首例新冠肺炎逝者遗体解剖报告：病变仍聚焦肺部 其他脏器损有待研究。2020年2月28日，在首例新冠肺炎逝者遗体解剖完成12天后，相关解剖报告公布。《报告》结果提示：新冠肺炎主要引起深部气道和肺泡损伤为特征的炎性反应，肺部纤维化及实变没有SARS导致的病变严重，而渗出性反应较SARS明显，对于心肌及心外膜、肾脏、脾脏、消化道器官、脑部是否存在与病毒感染相关的损害表现有待进一步研究。对于新冠肺炎逝者遗体的解剖报告，可以说，研究人员和临床医生已经等待已久。', '--category=25', '--keywords=News']
Uploading file...
<HttpError 403 "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.">

In [35]:
!ls -all

total 490740
drwxr-xr-x 1 root root      4096 Mar  2 03:28 .
drwxr-xr-x 1 root root      4096 Mar  2 02:17 ..
-rw-r--r-- 1 root root       429 Mar  2 02:41 client_secrets.json
drwxr-xr-x 1 root root      4096 Feb 19 17:12 .config
drwxr-xr-x 1 root root      4096 Feb  5 18:37 sample_data
-rw-r--r-- 1 root root  40388345 Mar  2 02:56 v020040f0000bpdrbf11h1vrb312um4g.mp4
-rw-r--r-- 1 root root   2384103 Mar  2 03:13 v02004200000bpdiml4pg620rt6sm8gg.mp4
-rw-r--r-- 1 root root  14793862 Mar  2 03:32 v02004630000bpdhj5hpjc2ptbbk6csg.mp4
-rw-r--r-- 1 root root  18921421 Mar  2 03:13 v02004630000bpdkkku8qbl46mgipr1g.mp4
-rw-r--r-- 1 root root 100351098 Mar  2 03:28 v02004730000bpdi944ttc5kktjn5020.mp4
-rw-r--r-- 1 root root   2896169 Mar  2 03:13 v020047e0000bpdivfc9hq5jogl4d4b0.mp4
-rw-r--r-- 1 root root  22982074 Mar  2 03:08 v020047e0000bpdn9cl81shluvkd9ka0.mp4
-rw-r--r-- 1 root root  96076594 Mar  2 02:55 v020047e0000bpds70nijqt1h4rqtnlg.mp4
-rw-r--r-- 1 root root   8852183 Mar  2 02:55 v0

# youtube 类目id

In [0]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.videoCategories.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/guides/code_samples#python

import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

def category():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "client_secrets.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

    request = youtube.videoCategories().list(
        part="snippet",
        regionCode="US"
    )
    response = request.execute()

    print(response)

category()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=898163829886-tn0jb3uua55fa2948lnbn5u9apt65npd.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=qbLuBwNB3iTFO3eEAn84kjjfD4Y2oa&prompt=consent&access_type=offline
Enter the authorization code: 4/xAGcbAb8BM_jQlL2TLOwH3TqgaNyFKwIrybY4EmiDt59ku-9LIxHiH4
{'kind': 'youtube#videoCategoryListResponse', 'etag': '"SJZWTG6xR0eGuCOh2bX6w3s4F94/Afw2jWDwrW-XewYrKFwgcGSSvVo"', 'items': [{'kind': 'youtube#videoCategory', 'etag': '"SJZWTG6xR0eGuCOh2bX6w3s4F94/Xy1mB4_yLrHy_BmKmPBggty2mZQ"', 'id': '1', 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Film & Animation', 'assignable': True}}, {'kind': 'youtube#videoCategory', 'etag': '"SJZWTG6xR0eGuCOh2bX6w3s4F94/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"', 'id': '2', 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Autos & V